<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Dummy-data" data-toc-modified-id="Dummy-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dummy data</a></span></li><li><span><a href="#Cross-validation" data-toc-modified-id="Cross-validation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cross-validation</a></span></li><li><span><a href="#Create-Tensorflow-DNN-model" data-toc-modified-id="Create-Tensorflow-DNN-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Tensorflow DNN model</a></span></li><li><span><a href="#Bayesian-optimisation" data-toc-modified-id="Bayesian-optimisation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Bayesian optimisation</a></span></li></ul></div>

# Nodos

## Imports

In [1]:
# General:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
import csv
import time
import shutil
import pickle
import logging

# Tensorflow:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K

# Sklearn
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error

# Scikit-Optimise
from skopt import gp_minimize, dump
from skopt.space import Categorical, Integer
from skopt.utils import use_named_args

# Statistics:
import scipy
from uncertainties import unumpy
import itertools
import statistics
itertools.imap = lambda *args, **kwargs: list(map(*args, **kwargs))

/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Dummy data

In [2]:
n_cols = 10
raw_data = pd.read_csv('/Users/wilsonwu/OUTBRAIK/dataset_1.csv', sep=';', decimal=',', index_col='Disease')
raw_data

,Region,Death cases,Infected cases,Period of time (months),Population density inhab/km3,Population,GDP (trillion USD),Infected rate,Mortality rate,Temperature C (1st month),Humidity %(1st month),Epidemic or not
Disease,,,,,,,,,,,,
2002 SARS China,Guangzhou,774.0,8090.0,8.0,1128.0,8392000.0,1471.0,0.10,9.57,19.4,64.0,1.0
2008 HFMD China,Fuyang,431.0,1520274.0,52.0,NaN,728000.0,NaN,"208,83",0.03,9.2,79.3,1.0
2009 Meningitis West Africa,NaN,1100.0,25000.0,3.0,NaN,NaN,NaN,#DIV/0!,4.40,NaN,NaN,1.0
2009 Influenza Mexico,Veracruz,395600.0,200000000.0,12.0,NaN,800000.0,NaN,"25000,00",0.20,NaN,NaN,1.0
2012 MERS Saudi Arabia,Jeddah,449.0,1123.0,36.0,NaN,NaN,NaN,#DIV/0!,39.98,NaN,NaN,1.0
2015 Influenza India,Rajasthan,2035.0,33761.0,4.0,NaN,NaN,NaN,#DIV/0!,6.03,NaN,NaN,1.0
2019 Measles Samoa,Samoa,83.0,5707.0,4.0,NaN,NaN,NaN,#DIV/0!,1.45,NaN,NaN,1.0
2019 Measles Tonga,Tongatapu,0.0,612.0,3.0,NaN,NaN,NaN,#DIV/0!,0.00,NaN,NaN,1.0
2019 Measles Rep. Of the Congo,Brazzaville,6000.0,250000.0,14.0,NaN,NaN,NaN,#DIV/0!,2.40,NaN,NaN,1.0


In [3]:
# Drop Region column and only use first row.
train_set = raw_data.drop('Region', axis=1).iloc[[0],:]
train_set

,Death cases,Infected cases,Period of time (months),Population density inhab/km3,Population,GDP (trillion USD),Infected rate,Mortality rate,Temperature C (1st month),Humidity %(1st month),Epidemic or not
Disease,,,,,,,,,,,
2002 SARS China,774.0,8090.0,8.0,1128.0,8392000.0,1471.0,0.10,9.57,19.4,64.0,1.0


In [4]:
for x in train_set.iloc[0,:].values.tolist():
    print(type(x))
    print(x)

<class 'numpy.float64'>
774.0
<class 'numpy.float64'>
8090.0
<class 'numpy.float64'>
8.0
<class 'numpy.float64'>
1128.0
<class 'numpy.float64'>
8392000.0
<class 'numpy.float64'>
1471.0
<class 'str'>
0.10
<class 'numpy.float64'>
9.57
<class 'numpy.float64'>
19.4
<class 'numpy.float64'>
64.0
<class 'numpy.float64'>
1.0


In [5]:
train_set.applymap(lambda x: str(x).replace(",", ".") if isinstance(x, float) else x)
train_set

,Death cases,Infected cases,Period of time (months),Population density inhab/km3,Population,GDP (trillion USD),Infected rate,Mortality rate,Temperature C (1st month),Humidity %(1st month),Epidemic or not
Disease,,,,,,,,,,,
2002 SARS China,774.0,8090.0,8.0,1128.0,8392000.0,1471.0,0.10,9.57,19.4,64.0,1.0


In [6]:
float_df = train_set.apply(pd.to_numeric).astype(float).sample(frac=1)

In [7]:
n = 10
for i in range(4):
    new_row = [x + n for x in float_df.iloc[0,:].values.tolist()]
    print(new_row)
    float_df = float_df.append(pd.DataFrame([new_row], columns=train_set.columns), sort=False)
float_df

[784.0, 8100.0, 18.0, 1138.0, 8392010.0, 1481.0, 10.1, 19.57, 29.4, 74.0, 11.0]
[784.0, 8100.0, 18.0, 1138.0, 8392010.0, 1481.0, 10.1, 19.57, 29.4, 74.0, 11.0]
[784.0, 8100.0, 18.0, 1138.0, 8392010.0, 1481.0, 10.1, 19.57, 29.4, 74.0, 11.0]
[784.0, 8100.0, 18.0, 1138.0, 8392010.0, 1481.0, 10.1, 19.57, 29.4, 74.0, 11.0]


,Death cases,Infected cases,Period of time (months),Population density inhab/km3,Population,GDP (trillion USD),Infected rate,Mortality rate,Temperature C (1st month),Humidity %(1st month),Epidemic or not
2002 SARS China,774.0,8090.0,8.0,1128.0,8392000.0,1471.0,0.1,9.57,19.4,64.0,1.0
0,784.0,8100.0,18.0,1138.0,8392010.0,1481.0,10.1,19.57,29.4,74.0,11.0
0,784.0,8100.0,18.0,1138.0,8392010.0,1481.0,10.1,19.57,29.4,74.0,11.0
0,784.0,8100.0,18.0,1138.0,8392010.0,1481.0,10.1,19.57,29.4,74.0,11.0
0,784.0,8100.0,18.0,1138.0,8392010.0,1481.0,10.1,19.57,29.4,74.0,11.0


In [8]:
float_df['Epidemic or no'] = [1.0, 1.0, 0.0, 1.0, 0.0]
float_df

,Death cases,Infected cases,Period of time (months),Population density inhab/km3,Population,GDP (trillion USD),Infected rate,Mortality rate,Temperature C (1st month),Humidity %(1st month),Epidemic or not,Epidemic or no
2002 SARS China,774.0,8090.0,8.0,1128.0,8392000.0,1471.0,0.1,9.57,19.4,64.0,1.0,1.0
0,784.0,8100.0,18.0,1138.0,8392010.0,1481.0,10.1,19.57,29.4,74.0,11.0,1.0
0,784.0,8100.0,18.0,1138.0,8392010.0,1481.0,10.1,19.57,29.4,74.0,11.0,0.0
0,784.0,8100.0,18.0,1138.0,8392010.0,1481.0,10.1,19.57,29.4,74.0,11.0,1.0
0,784.0,8100.0,18.0,1138.0,8392010.0,1481.0,10.1,19.57,29.4,74.0,11.0,0.0


## Cross-validation

In [9]:
def split_dataset(dataframe, n_splits):
    """Scikit-Learn KFold implementation for pandas DataFrame."""

    label_col = 'Epidemic or no'
    random_state = 2
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    kfolds = []
    global offset_col_name

    for train, validate in kf.split(dataframe):
        training = dataframe.iloc[train]
        train_labels = training[label_col]
        train_set = training.drop(label_col, axis=1)

        validating = dataframe.iloc[validate]
        validate_labels = validating[label_col]
        validate_set = validating.drop(label_col, axis=1)

        kfolds.append(
            [[train_set, validate_set],
             [train_labels, validate_labels]]
        )

    with open('/Users/wilsonwu/OUTBRAIK/outbraik/data/06_models/kfolds.json', "wb") as file:
        pickle.dump(kfolds, file)

    logging.info('Pickled kfolds nested list to JSON.')
    return kfolds

## Create Tensorflow DNN model

In [10]:
def create_model(num_dense_layers_base, num_dense_nodes_base,
                 num_dense_layers_end, num_dense_nodes_end,
                 activation, adam_b1, adam_b2, adam_eps):
    
    num_input_nodes = 11
    
    # Craete linear stack of layers.
    model = keras.Sequential()

    # Define input layer.
    model.add(keras.layers.Dense(
        num_input_nodes,  # N.umber of nodes
        input_shape=(num_input_nodes,)  # Tuple specifying data input dimensions only needed in first layer.
             ))

    # Define n number of hidden layers (base, i.e. first layers).
    for i in range(num_dense_layers_base):
        model.add(keras.layers.Dense(
            num_dense_nodes_base,
            activation=activation
        ))

    # Define n number of hidden layers (end, i.e. last layers).
    for i in range(num_dense_layers_end):
        model.add(keras.layers.Dense(
            num_dense_nodes_end,
            activation=activation
        ))

    # Add two output nodes.
    model.add(keras.layers.Dense(1, activation=keras.activations.linear))
    
    model.add(keras.layers.Activation('sigmoid'))

    # Define dam optimiser.
    optimizer = tf.keras.optimizers.Adam(
        lr=0.0001,  # Learning rate
        beta_1=adam_b1,  # Exponential decay rate for the first moment estimates.
        beta_2=adam_b2,  # Exponential decay rate for the second-moment estimates.
        epsilon=adam_eps  # Prevent any division by zero.
    )

    # Compile model.
    model.compile(
        loss='mae',  # Loss function
        optimizer=optimizer,  # Optimisaion function defined above.
        metrics=['mae']  # Metric to be recorded.
    )

    return model

## Bayesian optimisation

In [11]:
def train_model(fold, fold_num, n_calls, epochs):
    """
    1. Unpack training data.
    2. Define hyper-perameter ranges.
    3. Define early stopping perameters.
    4. Optimise hyper-perameters and save best model.
    5. Save mae per call to CSV.
    """
    logging.info('Training fold {}.'.format(str(fold_num)))
    
    # Retrieve data sets and convert to numpy array.
    train_X = fold[0][0].values
    validate_X = fold[0][1].values
    train_y = fold[1][0].values
    validate_y = fold[1][1].values

    # Define hyper-perameters.
    # Layers
    dim_num_dense_layers_base = Integer(low=1, high=2, name='num_dense_layers_base')
    dim_num_dense_nodes_base = Categorical(categories=list(np.linspace(5, 261, 10, dtype=int)),
                                           name='num_dense_nodes_base')
    dim_num_dense_layers_end = Integer(low=1, high=2, name='num_dense_layers_end')
    dim_num_dense_nodes_end = Categorical(categories=list(np.linspace(5, 261, 10, dtype=int)),
                                          name='num_dense_nodes_end')

    # Optimiser
    dim_adam_b1 = Categorical(categories=list(np.linspace(0.8, 0.99, 11)), name='adam_b1')
    dim_adam_b2 = Categorical(categories=list(np.linspace(0.8, 0.99, 11)), name='adam_b2')
    dim_adam_eps = Categorical(categories=list(np.linspace(0.0001, 0.5, 11)), name='adam_eps')

    dimensions = [dim_num_dense_layers_base, dim_num_dense_nodes_base,
                  dim_num_dense_layers_end, dim_num_dense_nodes_end,
                  dim_adam_b1, dim_adam_b2, dim_adam_eps]

    # Set early stopping variable to prevent overfitting.
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss',  # Monitor validation loss
        mode='min',  # Monitoring loss
        patience=20,  # Large patience for small batch size
        verbose=0)  # Do not output to terminal

    best_mae = np.inf
    
    # Start hyper-perameter optimisation.
    @use_named_args(dimensions=dimensions)
    def fitness(num_dense_layers_base, num_dense_nodes_base,
                num_dense_layers_end, num_dense_nodes_end,
                adam_b1, adam_b2, adam_eps):

        # Create the neural network with these hyper-parameters.
        model = create_model(num_dense_layers_base=num_dense_layers_base,
                             num_dense_nodes_base=num_dense_nodes_base,
                             num_dense_layers_end=num_dense_layers_end,
                             num_dense_nodes_end=num_dense_nodes_end,
                             activation=tf.keras.activations.relu,
                             adam_b1=adam_b1, adam_b2=adam_b2, adam_eps=adam_eps)

        history = model.fit(train_X, train_y, # Training data
                            epochs=epochs,  # Number of forward and backward runs.
                            validation_data=(validate_X, validate_y),  # Validation data
                            verbose=1,
                            callbacks=[early_stopping],  # Prevent overfitting.
                            batch_size=30)  # Increase efficiency

        mae = history.history['val_mae'][-1]
        # If the regressor accuracy of the saved model is improved...
        nonlocal  best_mae
        if mae < best_mae:
            # Save the new model to harddisk.
            model.save('/Users/wilsonwu/OUTBRAIK/outbraik/data/06_models/fold_' + str(fold_num) + '_model.h5')
            # Update the regressor accuracy.
            best_mae = mae

        # Delete the Keras model with these hyper-parameters from memory.
        del model

        # Clear the Keras session, otherwise it will keep adding new
        # models to the same TensorFlow graph each time we create
        # a model with a different set of hyper-parameters.
        K.clear_session()

        # Reset best MAE.
        best_mae = np.inf

        return mae

    # A place for optimiser to start looking.
    default_parameters = [2, 261, 1, 61, 0.857, 0.933, 0.20006]

    search_result = gp_minimize(func=fitness,
                                dimensions=dimensions,
                                acq_func='EI',  # Expected Improvement
                                n_calls=n_calls,
                                x0=default_parameters)

    # Save skopt object.
    dump(search_result,
         '/Users/wilsonwu/OUTBRAIK/outbraik/data/06_models/fold_' + str(fold_num) +  '_gp_minimize_result.pickle',
         store_objective=False)
    logging.info('Pickled fold {} Scikit-Optimise object.'.format(fold_num))

    logging.info('Fold {} final parameters: {}.'.format(str(fold_num), search_result.x))
    return search_result

In [12]:
def train_DNN(dataframe, n_splits, n_calls, epochs):
    
    kfolds = split_dataset(dataframe, n_splits)
    all_models = [train_model(fold, fold_num+1, n_calls, epochs) for fold_num, fold in enumerate(kfolds)]

    return all_models

In [ ]:
results = train_DNN(float_df, 5, 11, 3)

Train on 4 samples, validate on 1 samples
Epoch 1/3
4/4 [==============================] - 3s 627ms/sample - loss: 0.2500 - mae: 0.2500 - val_loss: 1.0000 - val_mae: 1.0000
Epoch 2/3
4/4 [==============================] - 0s 28ms/sample - loss: 0.2500 - mae: 0.2500 - val_loss: 1.0000 - val_mae: 1.0000
Epoch 3/3
4/4 [==============================] - 0s 31ms/sample - loss: 0.2500 - mae: 0.2500 - val_loss: 1.0000 - val_mae: 1.0000
Train on 4 samples, validate on 1 samples
Epoch 1/3
4/4 [==============================] - 2s 588ms/sample - loss: 0.7500 - mae: 0.7500 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 2/3
4/4 [==============================] - 0s 36ms/sample - loss: 0.7500 - mae: 0.7500 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 3/3
4/4 [==============================] - 0s 32ms/sample - loss: 0.7500 - mae: 0.7500 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Train on 4 samples, validate on 1 samples
Epoch 1/3
4/4 [==============================] - 2s 427ms/sample - l

In [ ]:
def plot_convergence(all_models, n_splits, n_calls):
    
    mae_logger = [[fold_num + 1, x] for fold_num, result in enumerate(all_models) for x in result['func_vals']]
    mae_df = pd.DataFrame(mae_logger, columns=['Fold', 'MAE (kcal/mol)'])
    
    # x values
    x = np.linspace(1, n_calls, n_calls)

    # y values
    mae = [mae_df.loc[mae_df.iloc[:, 0] == fold, 'MAE (kcal/mol)'].cummin()
           for fold in range(1, n_splits + 1)]
    cumm_mae = list(zip(*mae))
    y = [statistics.mean(call) for call in cumm_mae]

    # standard devation
    std = [statistics.stdev(call) for call in cumm_mae]

    # standard devation bounds
    y1 = [i - sd for i, sd in zip(y, std)]
    y2 = [i + sd for i, sd in zip(y, std)]

    # plot mean line
    fig, ax = plt.subplots(figsize=[8, 6])
    for axis in ['top','bottom','left','right']: ax.spines[axis].set_linewidth(2)

    ax.plot(x, y,
            color='green',
            linewidth=2,
            label='Average MAE over {} folds'.format(n_splits))

    # plot standard deviation fill bounds
    ax.fill_between(x, y1, y2,
                    fc='lightsteelblue',
                    ec='lightsteelblue',
                    label='Standard deviation')

    ax.set_xlabel('Number of calls $n$', fontsize=18)
    ax.set_ylabel('MAE / kcal mol$^{-1}$', fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=18)

    ax.legend(fontsize=18)
    plt.tight_layout()
    
    fig.savefig('/Users/wilsonwu/OUTBRAIK/outbraik/data/08_reporting/convergence_plot.png')
    
    return ax

In [ ]:
plot = plot_convergence(results, 5, 11)

In [ ]:
float_df

In [ ]:
test_X = np.asarray([[x + 10 for x in float_df.iloc[-1,:-1]]])
test_X

In [ ]:
def model_predict(model_num, test_entry):
    """Load model from HDF5 and return model prediction on a given test_entry."""

    model = tf.keras.models.load_model('/Users/wilsonwu/OUTBRAIK/outbraik/data/06_models/fold_' + str(model_num) + '_model.h5')

    return model.predict(test_entry)

In [ ]:
test_rst = model_predict(1, test_X)

In [ ]:
test_rst

In [ ]:
def create_SVC_model(param_C):
    """Returns a SVR class instance."""
    
    return SVC(gamma='auto',
               C=param_C,
               verbose=False)

In [ ]:
def train_SVC_model(fold, fold_num, n_calls):
    """
    1. Unpack data.
    2. Define fitness function for guassian process optmisation.
    3. Decorate fitness with pre-defined hyper-parameter ranges.
    4. Return and pickle final gp_minimize object.
    """
    logging.info('Started training fold {}...'.format(str(fold_num)))

    # Retrieve data sets and convert to numpy array
    train_X = fold[0][0].values
    validate_X = fold[0][1].values
    train_y = fold[1][0].values
    validate_y = fold[1][1].values
    
    # Set hyper-parameter ranges and append to list.
    dim_param_C = Categorical(categories=list(np.logspace(-3, 2, 6, dtype="float32")), name="param_C")

    # gp_minimize dimensions
    dimensions = [dim_param_C]
    
    best_mae = np.inf
    
    # Define function for gaussian process optimisation
    @use_named_args(dimensions=dimensions)
    def fitness(param_C):
        """Function for gaussian process optmisation."""

        # Create SVR model
        model = create_SVC_model(param_C)

        # Train model on training data
        model.fit(train_X, train_y)

        # Validate model
        predicted_y = model.predict(validate_X)
        mae = mean_absolute_error(validate_y, predicted_y)
        
        # Check if model improves
        nonlocal  best_mae
        if mae < best_mae:
            # Update new model accuracy.
            best_mae = mae
            # Overwrite model if mae improves
            pkl_file =  '/Users/wilsonwu/OUTBRAIK/outbraik/data/06_models/fold_' + str(fold_num) + 'SVC_model.pickle'
            with open(pkl_file, 'wb') as file: pickle.dump(model, file)
            logging.info('Saved {}.'.format(pkl_file))

        # Reset best MAE
        best_mae = np.inf
        
        return mae
    
    # Starting parameters
    default_parameters = [1.0]
    # Optimise hyper-parameters
    search_result = gp_minimize(func=fitness,
                                dimensions=dimensions,
                                acq_func='EI',  # Expected Improvement.
                                n_calls=n_calls,
                                x0=default_parameters,
                                verbose=False)

    # Save skopt object and analyse in a separate script as
    dump(search_result,
         '/Users/wilsonwu/OUTBRAIK/outbraik/data/06_models/fold_' + str(fold_num) +  'SVC_gp_minimize_result.pickle',
         store_objective=False)

    logging.info('Finished training fold {}: {}.'.format(str(fold_num), search_result.x))
    return search_result

In [ ]:
def train_SVC(dataframe, n_splits, n_calls):
    
    kfolds = split_dataset(dataframe, n_splits)
    SVC_results = [train_SVC_model(fold, fold_num + 1, n_calls) for fold_num, fold in enumerate(kfolds)]

    return SVC_results

In [ ]:
SVC_results = train_SVC(float_df, 5, 11)

In [ ]:
SVC_plot = plot_convergence(SVC_results, 5, 11)